# 가장 먼저 할 일! SparkSession !
가장 먼저 해야할 것은 SparkContext라는 스파크 객체를 만들어 주어야 한다. SparkContext를 만들어 주기 위해서 우선 SparkSession을 만들어 주자. 그리고 json 파일 형식으로 되어 있는 데이터를 읽어서 데이터의 스키마를 출력시켜보자.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .config('spark.some.config.option', 'some-value')\
        .getOrCreate()

### Create json file using spark
# SparkContext로 객체 생성
sc = spark.sparkContext

# json 파일 읽어들이기
path = './data/people.json'
peopleDF = spark.read.json(path)

# printSchema()로 json파일의 스키마 형태 볼 수 있음
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



# SQL로 DataFrame 형태로 데이터 출력해보기

In [5]:
peopleDF.createOrReplaceTempView("people")

# spark에서 제공하는 sql 메소드를 이용해 쿼리 날리기
# 쿼리문에서 people 테이블은 위에서 만들었던 view 테이블임!
teenagerNamesDF = spark.sql("select name from people where age between 13 and 19")
teenagerNamesDF.show()

+------+
|  name|
+------+
|Justin|
+------+



In [7]:
df = spark.read.json(path)

# global temporary view 생성
df.createOrReplaceGlobalTempView('people')

# 'global_temp' 라는 키워드 꼭 붙이기
sqlDF = spark.sql('select * from global_temp.people')
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
# 또한 데이터프레임은 RDD[String] 자료구조를 이용해서 json 데이터셋을 데이터프레임으로 만들 수 있음
jsonStrings = ['{"name": "Yin", "address":{"city":"Columbus", "state":"Ohio"}}']
# json -> RDD형식으로 만들기
otherPeopleRDD = sc.parallelize(jsonStrings)
# json파일 읽어오기
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



# Spark의 DataFrame의 칼럼에 접근해보자!

DataFrame Operations 라고도 하면 Untyped Dataset Operations 라고도 한다. Spark 내부에서 Dataset 과 DataFrame의 차이라고 한다면 Typed/Untyped 차이라고 한다. Typed dataset은 dataset으로 받아오는 데이터의 형태를 미리 정의해 놓은 것인 반면 Untyped dataset은 프로그램이 데이터의 형태를 추측해서 가져오는 것을 의미한다. 

In [10]:
df = spark.read.json(path)

df.select('name').show()

# 추가적으로 2개 이상의 컬럼을 추출하면서 기존의 컬럼에 연산을 가해 파생변수를 생성하여 추출하는 것도 가능하다.
df.select(df['name'], df['age']+1).show()

# age가 20보다 큰 데이터만 추출
df.filter(df['age'] > 20).show()

# age 컬럼으로 그룹핑 하고 데이터의 개수를 집계
df.groupBy('age').count().show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



# DataFrame의 Schema를 프로그래밍스럽게 명시해보자!

In [11]:
from pyspark.sql.types import *

# SparkContext 객체 생성
sc = spark.sparkContext

# txt file load
lines = sc.textFile('./data/people.txt')
parts = lines.map(lambda l: l.split(','))

## Step 1 -> value들 처리
# 각 라인을 tuple( , ) 형태로 convert
people = parts.map(lambda p: (p[0], p[1].strip())) # name에서 공백 strip

## Step 2 -> Schema들 처리
# 문자열로 인코딩된 스키마
schemaString = "name age"
# schemaString 요소를 loop돌면서 StructField로 만들기
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
# StructField 여러개가 있는 리스트를 StructType로 만들기
schema = StructType(fields)

## Step 3 -> value와 schema 활용해 Dataframe 생성
# 위에서 만든 schema를 RDD의 schema로 적용
schemaPeople = spark.createDataFrame(people, schema)

# View Table 생성해 쿼리 날려서 데이터 추출
schemaPeople.createOrReplaceTempView('people')
results = spark.sql('select * from people')
results.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+

